## Train OntoVAE model

In [2]:
# import packages
import sys
import scanpy as sc
sys.path.append('/workspace')
from sconto_vae.module.ontobj import *
from sconto_vae.module.utils import *
from sconto_vae.model.sconto_vae import *

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/opt/conda/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [3]:
# load ontobj
ontobj = Ontobj()
ontobj.load('/workspace/sconto_vae/data/GO/GO.ontobj')

In [4]:
# load anndata (Kang PBMC data - log normalized)
adata = sc.read_h5ad('/workspace/sconto_vae/data/train_pbmc.h5ad')

/opt/conda/lib/python3.10/site-packages/anndata/compat/__init__.py:298: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/opt/conda/lib/python3.10/site-packages/anndata/compat/__init__.py:298: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [5]:
# setup the anndata
adata = setup_anndata_ontovae(adata, ontobj)

In [5]:
# initialize the model
model = scOntoVAE(adata)
model.to(model.device)

scOntoVAE(
  (encoder): Encoder(
    (encoder): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=19469, out_features=1755, bias=True)
        (1): BatchNorm1d(1755, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Dropout(p=0.2, inplace=False)
        (3): ReLU()
      )
    )
    (mu): Sequential(
      (0): Linear(in_features=1755, out_features=1755, bias=True)
      (1): Dropout(p=0.5, inplace=False)
    )
    (logvar): Sequential(
      (0): Linear(in_features=1755, out_features=1755, bias=True)
      (1): Dropout(p=0.5, inplace=False)
    )
  )
  (decoder): OntoDecoder(
    (decoder): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=1755, out_features=42, bias=True)
      )
      (1): Sequential(
        (0): Linear(in_features=1797, out_features=213, bias=True)
      )
      (2): Sequential(
        (0): Linear(in_features=2010, out_features=621, bias=True)
      )
      (3): Sequential(
        (0): Linear(in_featur

In [8]:
# train the model
model.train_model('/workspace/sconto_vae/test/best_model.pt',   
                     lr=1e-4,                                 
                     kl_coeff=1e-4,                           
                     batch_size=128,                          
                     epochs=5)      

Epoch 1 of 5


100%|██████████| 27/27 [00:00<00:00, 105.00it/s]


New best model!
Train Loss: 38731.7956
Val Loss: 28209.4666
Epoch 2 of 5


100%|██████████| 27/27 [00:00<00:00, 103.26it/s]


New best model!
Train Loss: 27007.4509
Val Loss: 21975.3391
Epoch 3 of 5


...

New best model!
Train Loss: 24083.1830
Val Loss: 20569.4857
Epoch 4 of 5


100%|██████████| 27/27 [00:00<00:00, 103.08it/s]


New best model!
Train Loss: 22951.9595
Val Loss: 19957.0698
Epoch 5 of 5


100%|██████████| 27/27 [00:00<00:00, 103.38it/s]


New best model!
Train Loss: 22134.4548
Val Loss: 19398.7668


In [6]:
# load the best model
model.load('/workspace/sconto_vae/test')

In [7]:
# compute pathway activities
act = model.get_pathway_activities()

/opt/conda/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)
